In [5]:
import pandas as pd

# Charger le fichier CSV
file_path = "C:/Users/HADIL MARAI/Downloads/axeone.csv"
df = pd.read_csv(file_path)

# Afficher les premières lignes pour comprendre la structure
df.head()


,Organisation,Official_Link,Opportunities_Link
0,European Union (EU),https://europa.eu/,https://research-and-innovation.ec.europa.eu/f...
1,USAID,https://www.usaid.gov,https://oig.usaid.gov/news
2,World bank,https://www.worldbank.org,https://projects.worldbank.org/en/projects-ope...
3,GIZ (German Dev. Agency),https://www.giz.de,https://www.giz.de/de/mit_der_giz_arbeiten/421...
4,EBRD,https://www.ebrd.com/,https://www.ebrd.com/small-business-support/fl...


In [6]:
df['Opportunities_Link']

0    https://research-and-innovation.ec.europa.eu/f...
1                           https://oig.usaid.gov/news
2    https://projects.worldbank.org/en/projects-ope...
3    https://www.giz.de/de/mit_der_giz_arbeiten/421...
4    https://www.ebrd.com/small-business-support/fl...
5    https://www.isdb.org/communityoutreach/project...
6    https://www.educationaboveall.org/our-work/glo...
7              https://www.afdb.org/en/news-and-events
8    https://opecfund.org/what-we-offer/grants/gran...
9    https://www.rockefellerfoundation.org/fellowsh...
Name: Opportunities_Link, dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Organisation        10 non-null     object
 1   Official_Link       10 non-null     object
 2   Opportunities_Link  10 non-null     object
dtypes: object(3)
memory usage: 368.0+ bytes


## Scraping avec requests + BeautifulSoup 
### Test sur un seul site

### WEB DETECTING THE OPEN AND FORTHCOMINF PROJECTS WITH THEIR LINK

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
service = Service(r"C:\Users\HADIL MARAI\Desktop\ODF\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

start_url = "https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/calls-for-proposals?pageNumber=1&pageSize=50&sortBy=startDate&status=31094501,31094502"
driver.get(start_url)

with open("projets_EU_All.txt", "w", encoding="utf-8") as f:
    for page_num in range(1, 9):  # Pages 1 to 50
        try:
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.cft-call-list-table-title-link, a[href*='topic-details'], a[href*='competitive-calls-cs']"))
            )
        except Exception:
            print(f"❌ Aucun projet trouvé sur la page {page_num}.")
            break

        time.sleep(2)
        
        project_links = driver.find_elements(
            By.CSS_SELECTOR,
            "a.cft-call-list-table-title-link, a[href*='topic-details'], a[href*='competitive-calls-cs']"
        )

        print(f"📄 Page {page_num} : {len(project_links)} projets trouvés.")

        for link in project_links:
            titre = link.text.strip()
            href = link.get_attribute("href")

            try:
                container = link.find_element(By.XPATH, "./ancestor::eui-card")
            except:
                container = None

            # --- Main Status ---
            try:
                status_element = container.find_element(
                    By.CSS_SELECTOR, "span.eui-u-text-nowrap.eui-label"
                )
                status = status_element.text.strip()
            except:
                status = "Statut inconnu"

            # --- Dates and Additional Status ---
            opening_date = "Non spécifiée"
            next_deadline = "Non spécifiée"
            additional_status = "Non spécifié"

            try:
                result_card = container.find_element(
                    By.CSS_SELECTOR, "sedia-result-card-type.eui-u-display-block.eui-u-mt-xs.ng-star-inserted"
                )

                # Get all <strong> elements with date values
                strongs = result_card.find_elements(By.CSS_SELECTOR, "strong.ng-star-inserted")
                if len(strongs) >= 1:
                    opening_date = strongs[0].text.strip()
                if len(strongs) >= 2:
                    next_deadline = strongs[1].text.strip()

                # Get the last <span> that is not a "|" separator
                spans = result_card.find_elements(By.CSS_SELECTOR, "span.ng-star-inserted")
                for span in reversed(spans):
                    span_text = span.text.strip()
                    if span_text and span_text != "|":
                        additional_status = span_text
                        break

            except:
                pass

            # --- Output Results ---
            if titre and href and href.startswith("http"):
                f.write(
                    f"Titre: {titre}\n"
                    f"Lien: {href}\n"
                    f"Statut: {status}\n"
                    f"Date d'ouverture: {opening_date}\n"
                    f"Date limite: {next_deadline}\n"
                    
                )
                print(
                    f"✔ {titre} | {href} | Statut: {status} | "
                    f"📅 Ouverture: {opening_date} → Deadline: {next_deadline} | Extra: {additional_status}"
                )








        # Try clicking the next page icon
        try:
            next_icon = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'eui-icon-svg[aria-label="Go to next page"]'))
            )

            # Check if it's disabled
            is_disabled = next_icon.get_attribute("aria-disabled")
            if is_disabled == "true":
                print("✅ Fin de la pagination (bouton désactivé).")
                break

            # Scroll into view and click using JavaScript
            driver.execute_script("arguments[0].scrollIntoView(true);", next_icon)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", next_icon)

            print("➡️ Passage à la page suivante...")
            time.sleep(3)

            # Optional cache cleanup
            driver.delete_all_cookies()
            driver.execute_script("window.localStorage.clear();")
            driver.execute_script("window.sessionStorage.clear();")

        except Exception as e:
            print(f"⚠️ Erreur lors de la tentative de passage à la page suivante : {e}")
            break

driver.quit()


📄 Page 1 : 46 projets trouvés.
✔ TALTECH MERIT Project Mobility Grant | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/competitive-calls-cs/10822?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 01 September 2025 → Deadline: 01 October 2025 | Extra: Multiple Cut-off
✔ TALTECH MERIT Project Excellence Scholarship Award | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/competitive-calls-cs/10821?isExactMatch=true&status=31094501,31094502&order=DESC&pageNumber=1&pageSize=50&sortBy=startDate | Statut: Forthcoming | 📅 Ouverture: 01 September 2025 → Deadline: 01 October 2025 | Extra: Multiple Cut-off
✔ Waste-to-value devices - circular production of renewable fuels, chemicals and materials | https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/opportunities/topic-details/HORIZON-EIC-2025-PATHFINDERCHALLENGES-01-04?isExac